In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('unsloth/Llama-3.2-1B-Instruct')

In [4]:
streamer = TextStreamer(tokenizer)

In [5]:
ori_model = AutoModelForCausalLM.from_pretrained(
    'unsloth/Llama-3.2-1B-Instruct', tie_word_embeddings=False, torch_dtype = torch.bfloat16,
).cuda()

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
ori_model.lm_head.weight.data = ori_model.model.embed_tokens.weight.data.clone()

In [7]:
state_dict = ori_model.state_dict()

In [8]:
from safetensors import safe_open

f = safe_open(f"lora-embedding-256-llama3.2-1b-small-malaysian/checkpoint-17300/adapter_model.safetensors", framework="pt", device=0)
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [9]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A).type(W.dtype)
    B = f.get_tensor(B).type(W.dtype)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|██████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 513.13it/s]


In [10]:
d = [
    {'role': 'user', 'content': 'ape isu dengan najib'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.0,
    do_sample=False,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

ape isu dengan 

/home/husein/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/husein/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


najib<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Terima kasih atas pertanyaan anda. Saya akan cuba memberikan jawapan yang terperinci dan jelas.

Isu Najib Razak adalah berhubung dengan skandal 1Malaysia Development Ber


In [11]:
d = [
    {'role': 'user', 'content': 'kwsp tu ape'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.0,
    do_sample=False,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

kwsp tu ape<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Kwsp merujuk kepada "Kumpulan Wang Simpanan Pekerja" yang merupakan skim simpanan persaraan yang ditawarkan oleh kerajaan Malaysia. Ia bertujuan untuk membantu


In [12]:
d = [
    {'role': 'user', 'content': 'camne nk code react, reply in jawi'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=False,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react, reply in jawi<|eot_id|><|start_header_id|>assistant<|end_header_id|>

اونتوق ممبوات کومڤونن React يڠ مڽوکوڠ 

/home/husein/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


فوڠسي "کليک"، اندا بوليه مڠݢوناکن کوم


In [13]:
d = [
    {'role': 'user', 'content': 'camne nk code react website order kasut, reply in sabah'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=False,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react website order kasut, reply in sabah<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bah, kunuk sia mau kasi tau macam mana mo bikin laman web untuk pesanan kasut pakai React. Tingu saja la:

1. Mula-mula, kita mesti pasang dulu


In [14]:
d = [
    {'role': 'user', 'content': 'You are going to enter reasoning mode, first you try to think step-by-step in kelantan after that give the final answer.'},
    {'role': 'user', 'content': 'Derive the joint density function of the DE and SCF biases (f(DE, SCF)) using a Bayesian approach, assuming that the parameters α, β, λ, and θ follow normal distributions with known means and variances, and that the error terms ε and η follow normal distributions with zero means and unknown variances. Use the derived joint density function to compute the expected value of the portfolio return (E(S_t)) and the expected value of the portfolio volatility (E(σ)) across all investors.'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are going to enter reasoning mode, first you try to think step-by-step in kelantan after that give the final answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Derive the joint density function of the DE and SCF biases (f(DE, SCF)) using a Bayesian approach, assuming that the parameters α, β, λ, and θ follow normal distributions with known means and variances, and that the error terms ε and η follow normal distributions with zero means and unknown variances. Use the derived joint density function to compute the expected value of the portfolio return (E(S_t)) and the expected value of the portfolio volatility (E(σ)) across all investors.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Dalam senario ini, kita akan menggunakan pendekatan Bayesian untuk merumuskan taburan kebarangkalian fungsi ket

In [16]:
tokenizer.push_to_hub('malaysia-ai/Malaysian-Llama-3.2-1B')

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/malaysia-ai/Malaysian-Llama-3.2-1B/commit/80d1e84cbf1b8f3c72eba6e7e67d2c23f10f1c85', commit_message='Upload tokenizer', commit_description='', oid='80d1e84cbf1b8f3c72eba6e7e67d2c23f10f1c85', pr_url=None, repo_url=RepoUrl('https://huggingface.co/malaysia-ai/Malaysian-Llama-3.2-1B', endpoint='https://huggingface.co', repo_type='model', repo_id='malaysia-ai/Malaysian-Llama-3.2-1B'), pr_revision=None, pr_num=None)

In [17]:
ori_model.push_to_hub('malaysia-ai/Malaysian-Llama-3.2-1B')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

[2025-02-07 08:09:46,252] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


model.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/malaysia-ai/Malaysian-Llama-3.2-1B/commit/ce72cb898e35d1e0fe399557de9407aad4afb602', commit_message='Upload LlamaForCausalLM', commit_description='', oid='ce72cb898e35d1e0fe399557de9407aad4afb602', pr_url=None, repo_url=RepoUrl('https://huggingface.co/malaysia-ai/Malaysian-Llama-3.2-1B', endpoint='https://huggingface.co', repo_type='model', repo_id='malaysia-ai/Malaysian-Llama-3.2-1B'), pr_revision=None, pr_num=None)